In [1]:
import pytube
from pytube import YouTube
import os

from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

In [2]:
def download_video(url, video_id, person, dl_path, ext='.mp4'):
    """
    Downloads the videos and cut the segments
    """
    file_extension='mp4' if ext==".mp4" else None
    filename = f"{person}-{video_id}"
    
    yt = YouTube(url+video_id)
    try:
        #video = yt.streams.filter(progressive=True, file_extension=file_extension).order_by('resolution').desc().first()
        video = yt.streams.filter(progressive=True, file_extension=file_extension).get_highest_resolution()
        video.download(filename=filename+ext, output_path=dl_path)
    except pytube.exceptions.RegexMatchError:
        print("RegexMatchError: Could not find match for multiple. Skipping this video.")


In [5]:
download_video("https://www.youtube.com/watch?v=", 'OuhBF74ZD8E', 'trump', 'Trump/')

In [6]:
# Trump
14+11+8+17+10+6+13+12

91

In [12]:
def extract_audio(video_path, output_wav_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio

    audio_clip.write_audiofile(output_wav_path)

    video_clip.close()
    audio_clip.close()

In [13]:
extract_audio('Eisenhower/kamyar-eisenhower.mp4', 'Eisenhower/kamyar-eisenhower.wav')

MoviePy - Writing audio in Eisenhower/kamyar-eisenhower.wav


MoviePy - Done.


In [11]:


def trim_video_and_audio(input_video_path, input_audio_path, output_folder):
    video_clip = VideoFileClip(input_video_path)
    audio_clip = AudioFileClip(input_audio_path)

    segment_length = 10  # in seconds
    total_duration = min(video_clip.duration, audio_clip.duration)
    num_segments = int(total_duration // segment_length)

    os.makedirs(output_folder, exist_ok=True)

    for i in range(num_segments):
        start_time = i * segment_length
        end_time = (i + 1) * segment_length
        output_video_path = os.path.join(output_folder, f"segment_{i + 1}.mp4")
        output_audio_path = os.path.join(output_folder, f"segment_{i + 1}.wav")

        video_segment = video_clip.without_audio().subclip(start_time, end_time)
        audio_segment = audio_clip.subclip(start_time, end_time)

        video_segment.write_videofile(output_video_path, codec="libx264")
        audio_segment.write_audiofile(output_audio_path)

    video_clip.close()
    audio_clip.close()




In [ ]:
input_video_path = "Eisenhower/eisenhower-K_5Lgf5GjIQ.mp4"  # Replace with the path to your input video
input_audio_path = "Eisenhower/kamyar-eisenhower.wav"  # Replace with the path to your input audio
output_folder = "Eisenhower/segments/"  # Folder where output segments will be saved

trim_video_and_audio(input_video_path, input_audio_path, output_folder)

# Extract a section of video

In [5]:
def extract_section_video(input_video_path, start, end, output_folder):
    video_clip = VideoFileClip(input_video_path)

    os.makedirs(output_folder, exist_ok=True)

    output_video_path = os.path.join(output_folder, f"segment_{start}_to_{end}.mp4")

    video_segment = video_clip.without_audio().subclip(start, end)

    video_segment.write_videofile(output_video_path, codec="libx264")

    video_clip.close()

In [12]:
input_video_path = "Kennedy/kennedy-58O2De-iPOk.mp4"  # Replace with the path to your input video
output_folder = "Kennedy/"  # Folder where output segments will be saved

extract_section_video(input_video_path, 11*60+48,  13*60+47,output_folder)

Moviepy - Building video Kennedy/segment_708_to_827.mp4.
Moviepy - Writing video Kennedy/segment_708_to_827.mp4



Moviepy - Done !
Moviepy - video ready Kennedy/segment_708_to_827.mp4


# Sync audio files and a longer video that should be segmentized

In [7]:
import numpy as np
import os
import pytube
from pytube import YouTube
import os

from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

def sync_video_audio(video_path, audio_folder, output_folder):
    video_clip = VideoFileClip(video_path)
    audio_paths = sorted(os.listdir(audio_folder))
    audio_lens = [AudioFileClip(os.path.join(audio_folder, audio_path)).duration for audio_path in audio_paths]
    total_audio_duration = sum(audio_lens)

    if total_audio_duration > video_clip.duration:
        raise ValueError(f"Total Audio length is {sum(audio_lens)}, which is larger than the input video with length of {video_clip.duration}")
        # # Calculate the factor by which to extend the video duration
        # duration_factor = total_audio_duration / video_clip.duration

        # # Duplicate video frames to match audio duration
        # print(video_clip.duration * np.linspace(0, 1, int(duration_factor)))
        # duplicated_frames = [video_clip.get_frame(int(t)) for t in video_clip.duration * np.linspace(0, 1, int(duration_factor))]

        # # Create a video clip from duplicated frames
        # extended_video_clip = VideoFileClip(None, fps_source=video_clip.fps)
        # extended_video_clip = extended_video_clip.set_make_frame(lambda t: duplicated_frames[int(t * video_clip.fps)])


    
    # Create output folder if not existing
    os.makedirs(output_folder, exist_ok=True)
    # Remove prec output files
    filelist = [ f for f in os.listdir(output_folder) if f.endswith(".mp4") ]
    for f in filelist:
        os.remove(os.path.join(output_folder, f))
    elapsed_video = 0

    for i, len in enumerate(audio_lens):
        audio_clip = AudioFileClip(os.path.join(audio_folder, audio_paths[i]))

        
        start_time = elapsed_video
        end_time = start_time + len
        output_video_path = os.path.join(output_folder, f"segment_{os.path.splitext(audio_paths[i])[0]}.mp4")

        video_segment = video_clip.without_audio().subclip(start_time, end_time)

        video_segment.write_videofile(output_video_path, codec="libx264")
        elapsed_video += len

    video_clip.close()
    # extended_video_clip.close()
    audio_clip.close()

In [14]:
video_path = "./Demo/Trump/video_orig/trump-highres.mp4"
audio_folder = "./Demo/Trump/audio_cloned/"
output_folder = "./Demo/Trump/segments_cloned/"

sync_video_audio(video_path, audio_folder, output_folder)

Moviepy - Building video ./Demo/Trump/segments_cloned/segment_1.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_1.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_1.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_2.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_2.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_2.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_3.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_3.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_3.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_4.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_4.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_4.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_5.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_5.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_5.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_6.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_6.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_6.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_7.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_7.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_7.mp4
Moviepy - Building video ./Demo/Trump/segments_cloned/segment_8.mp4.
Moviepy - Writing video ./Demo/Trump/segments_cloned/segment_8.mp4



Moviepy - Done !
Moviepy - video ready ./Demo/Trump/segments_cloned/segment_8.mp4


# Concatenate multiple videos

In [39]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os

def concatenate_videos_in_folder(folder_path, output_path):
    video_files = []
    
    # Get a list of video files in the folder
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".mp4"):
            video_files.append(os.path.join(folder_path, file))
    
    if not video_files:
        print("No video files found in the folder.")
        return
    
    # Load video clips
    video_clips = [VideoFileClip(file) for file in video_files]
    
    # Calculate total duration of each video clip
    durations = [clip.duration for clip in video_clips]
    
    # Concatenate video clips based on their duration
    final_clip = concatenate_videoclips(video_clips, method="compose")
    
    # Write the concatenated video to the output path
    final_clip.write_videofile(output_path, codec="libx264")

In [40]:
input_folder = "results/"  # Replace with the path to the folder with videos
output_video = "output_stitched_kennedy.mp4"   # Replace with the desired output video path

concatenate_videos_in_folder(input_folder, output_video)


chunk:   0%|          | 1/2573 [01:44<74:18:54, 104.02s/it, now=None]

Moviepy - Building video output_stitched_kennedy.mp4.
MoviePy - Writing audio in output_stitched_kennedyTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 1/2573 [01:44<74:57:33, 104.92s/it, now=None]

MoviePy - Done.
Moviepy - Writing video output_stitched_kennedy.mp4



chunk:   0%|          | 1/2573 [01:48<77:11:06, 108.04s/it, now=None]

Moviepy - Done !
Moviepy - video ready output_stitched_kennedy.mp4
